In [1]:
# use our model Eval 1800 test dataset(289) 
import numpy as np
from PIL import Image
import sys
sys.path.insert(0,'./caffe/python')
import caffe
import os

import scipy.io as sio    
import matplotlib.pyplot as plt  
#%matplotlib inline
plt.rcParams['figure.figsize']=[15,15]
import time
import scipy.misc

INPUT_CHANNEL = 3
MEAN = [114.578, 115.294, 108.353]
DATASET = 'douyu_7100'
n_cl = 2

# load net
DEPLOY = './experiment/exper_douyu2700_googlenet/deploy.prototxt'
CAFFEMODEL = './experiment/exper_douyu2700_googlenet/snapshot/douyu2700_train_without_bn_iter_130000.caffemodel'
net = caffe.Net(DEPLOY, CAFFEMODEL, caffe.TEST)

#imglist = open('./data/'+DATASET+'/test.txt').readlines()
imglist = sio.loadmat('../../[EG_code_data]_release/data/testlist.mat')['testlist']
datanum = len(imglist)

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n,n)

hist = 0                   

allIoU = 0;
numimg = 0;
for img in imglist[0]:  
    if os.path.exists('../../[EG_code_data]_release/data/portraitFCN_data/%05d.mat'%img):
        im = sio.loadmat('../../[EG_code_data]_release/data/portraitFCN_data/%05d.mat'%img)['img']  
        numimg = numimg + 1

        in_ = np.array(im, dtype=np.float32)
        #in_ = in_[:,:,::-1]
        in_ = in_.transpose((2,0,1))

        # load label
        label = sio.loadmat('../../[EG_code_data]_release/data/images_mask/%05d_mask.mat'%img)['mask']
        label[label!=0]==1

        if INPUT_CHANNEL==3:
            imshape = (600, 800, 3)
            in_batch = np.ndarray(shape=(1, imshape[2], imshape[1], imshape[0]), dtype=np.float32)
            in_batch[0,0:3,:,:] = in_.copy()
        start = time.clock()
        # shape for input (data blob is N x C x H x W), set data
        net.blobs['data'].reshape(*in_batch.shape)
        net.blobs['data'].data[...] = in_batch

        # run net and take argmax for prediction
        net.forward()
        out = net.blobs['score'].data[0].argmax(axis=0)
        
        this_hist =  fast_hist(label.flatten(),out.flatten(),n_cl)
        this_hist = np.array(this_hist, dtype=np.float32)
        this_iu = np.diag(this_hist) / (this_hist.sum(1) + this_hist.sum(0) - np.diag(this_hist))
        if np.isnan(this_iu[0]):
            this_iu[0] = 1
        if np.isnan(this_iu[1]):
            this_iu[1] = 1
        allIoU += this_iu[1];
        print img, this_iu, np.mean(this_iu) # first is neg iou, second is pos iou 
        hist += this_hist
            
hist = np.array(hist, dtype=np.float32)
iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
print '================================= result ================================='
print 'INPUT_CHANNEL: ',INPUT_CHANNEL
print 'DEPLOY: ',DEPLOY
print 'CAFFEMODEL: ',CAFFEMODEL
print 'TEST DATASET: ',DATASET,
print 'TEST DATASET NUM: ',datanum
print 'IOU: ', iu, np.mean(iu)    
print 'allIOU: ', float(allIoU)/numimg, numimg




2 [ 0.37952718  0.7123251 ] 0.545926
5 [ 0.79679716  0.89722824] 0.847013
8 [ 0.68164444  0.82422841] 0.752936
11 [ 0.78849703  0.72890371] 0.7587
12 [ 0.62971139  0.85110241] 0.740407
14 [ 0.74180162  0.91641456] 0.829108
16 [ 0.73133087  0.79413426] 0.762733
24 [ 0.7161389   0.93739372] 0.826766
37 [ 0.62138188  0.81405026] 0.717716
40 [ 0.34578544  0.72508603] 0.535436
44 [ 0.70803469  0.90446228] 0.806248
55 [ 0.69684327  0.82098925] 0.758916
59 [ 0.49969772  0.78158253] 0.64064
61 [ 0.92689455  0.901214  ] 0.914054
62 [ 0.6765902   0.79602599] 0.736308
64 [ 0.86139888  0.87646759] 0.868933
65 [ 0.6642831   0.82922715] 0.746755
73 [ 0.91008443  0.89606923] 0.903077
75 [ 0.91028297  0.97373319] 0.942008
76 [ 0.61934102  0.67220157] 0.645771
79 [ 0.76591152  0.87359673] 0.819754
80 [ 0.79837036  0.77758884] 0.78798
83 [ 0.6350978   0.78342038] 0.709259
85 [ 0.49021554  0.77806962] 0.634143
93 [ 0.58178598  0.68855667] 0.635171
94 [ 0.71437448  0.84613341] 0.780254
96 [ 0.5625537   0.

KeyboardInterrupt: 